In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

In [ ]:
def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [ ]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    
    # tensorflow가 인풋으로 받은 shape는 [batch_size, height, width, channel]인데
    # mnist의 경우 graysclae image이기 때문에 채널이 1인 것이 생략되어서 3개의 shape
    # 그래서 채널을 하나 더 만들어줘야 함
    train_data = np.expand_dims(train_data, axis=-1) #[N,28,28] -> [N,28,28,1]
    # axis=-1은 여기서 axis=3과 같음. (맨 끝에 채널을 붙여달라는 말이므로)    
    test_data = np.expand_dims(test_data, axis=-1) #[N,28,28] -> [N,28,28,1]
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]

    # one-hot-encoding
    train_labels = to_categorical(train_labels, 10) #[N,] -> [N,10]
    # label 개수가 10개 이므로 10을 넣음
    test_labels = to_categorical(test_labels, 10) #[N,] -> [N,10]


In [2]:
# shape 펼쳐주기
def flatten():
    return tf.keras.layers.Flatten()

# fully connected layer
def dense(channel, weight_init):
    return tf.keras.layers.Dense(units=channel, use_bias = True,
                                 kernel_initialier=weight_init)
# units=channel : output으로 나가는 채널을 몇개로 설정한 것인지
# use_bias : bias를 사용할 지
# kernel_initialier : weight initializer

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

방법1) 클래스로 짜기

In [ ]:
class create_model(tf.keras.Model):
    def __init__(self, label_dim): # label_dim: 최종적으로 몇 개의 아웃풋을 낼 건지
        super(create_model, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal() # 평균이 0, 분산이 1
        
        self.model = tf.keras.Sequential() 
        # convolution이나 fully connected를 층층이 쌓아가는(리스트에 더해주는) 과정
        # list 자료구조 타입
        
        self.model.add(flatten()) # [N,28,28,1] -> [N, 784] # full connected를 할 거기 때문에
        
        # for문 코드 : 총 4층
        for i in range(2):
            # [N,784] -> [N,256] -> [N,256]
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim,weight_init)) # [N,256] -> [N,10]

In [ ]:
def call(self, x, training=None, mask=None):
    x = self.model(x)
    return x

방법2) 함수로 짜기

In [ ]:
# def create_model(label_dim):
#     weight_init = tf.keras.initializers.RandomNormal()

#     model= tf.keras.Sequential()
#     model.add(flatten())

#     for i in range(2):
#         model.add(dense(256, weight_init))
#         model.add(relu())
    
#     model.add(dense(label_dim,weight_init))

#     return model